<a href="https://colab.research.google.com/github/Twinkle0799/Predicton-of-House-prices-using-tensorflow/blob/master/Tensorflow%20house%20price%20predicting%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>